In [1]:
# python 11에서 발생하는 버그 수정
import inspect

if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec

In [2]:
from net import TransPoseNet
import torch

# 모델 인스턴스화
model = TransPoseNet()
model.eval()  # 반드시 eval 모드로 전환

c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\net.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(paths.weights_f

TransPoseNet(
  (pose_s1): RNN(
    (rnn): LSTM(256, 256, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=72, out_features=256, bias=True)
    (linear2): Linear(in_features=512, out_features=15, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pose_s2): RNN(
    (rnn): LSTM(64, 64, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=87, out_features=64, bias=True)
    (linear2): Linear(in_features=128, out_features=69, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pose_s3): RNN(
    (rnn): LSTM(128, 128, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=141, out_features=128, bias=True)
    (linear2): Linear(in_features=256, out_features=90, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (tran_b1): RNN(
    (rnn): LSTM(64, 64, num_layers=2, batch_first=True, bidirectional=True)
    (linear1): Linear(in_features=87, out_features=64, bi

### ONNX 사용

In [3]:
# 더미 입력 생성 (예: [batch_size, input_size])
# TransPoseNet에서 필요한 IMU 입력 크기를 확인하여 더미 입력 생성
# dummy_input = torch.randn(1, 6 * 3 + 6 * 9)  # 예시: (1, IMU 크기)
dummy_input = (torch.randn(1, 6 * 3), torch.randn(1, 6 * 9))  # 예시: (1, IMU 크기)

# ONNX로 변환
onnx_file_path = "transpose_net_241217.onnx"
torch.onnx.export(
    model,  # PyTorch 모델
    dummy_input,  # 예제 입력
    onnx_file_path,  # 저장할 ONNX 파일 경로
    export_params=True,  # 가중치를 함께 저장
    opset_version=11,  # ONNX Opset 버전 (최소 11 이상 권장)
    input_names=["acc", "ori"],  # 입력 이름
    output_names=["pose", "tran"],  # 출력 이름
    dynamic_axes={
        "acc": {0: "batch_size"},  # 배치 크기 동적 지원
        "ori": {0: "batch_size"},
        "pose": {0: "batch_size"},
        "tran": {0: "batch_size"},
    }
)
print(f"ONNX 모델이 {onnx_file_path}에 저장되었습니다.")

c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\net.py:176: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  lfoot_pos, rfoot_pos = art.math.forward_kinematics(pose[joint_set.lower_body].unsqueeze(0),
c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\net.py:181: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  indices = torch.tensor([7, 8])  # 7:9에 해당하는 인덱스
c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\net.py:190: TracerWarning: Converting a tensor to a Python number might cause the trace to be inco

torch.Size([24, 3, 3])


c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\.venv\Lib\site-packages\numpy\core\shape_base.py:212: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  return tuple(arrays)
c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\.venv\Lib\site-packages\numpy\core\shape_base.py:212: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  return tuple(arrays)
c:\Users\sage\SynologyDrive\dev\transpose_livestream\model\TransPose\.venv\Lib\site-packages\numpy\core\shape_base.py:443: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of

ONNX 모델이 transpose_net_241217.onnx에 저장되었습니다.


### ONNX 경량화 모델

In [ ]:
from onnxsim import simplify
import onnx

# ONNX 모델 로드
onnx_model = onnx.load("transpose_net_241217.onnx")

# Simplify 모델
# simplified_model, check = simplify(onnx_model)

# Simplify 모델, 검증 비활성화
simplified_model, check = simplify(onnx_model, skip_fuse_bn=True, check_n=False)

# 단순화 후 저장
onnx.save(simplified_model, "simplified_model_241217.onnx")


### Torch 모델 (실패)

In [4]:
# Dummy Input: IMU 데이터 (예: [batch_size, n_imu])
# dummy_input = torch.randn(1, 69)  # Replace 69 with your model's input size

# TorchScript 변환
scripted_model = torch.jit.script(model)

# 모델 저장
scripted_model.save("transpose_net.pt")
print("TorchScript 모델이 'transpose_net.pt'로 저장되었습니다.")

RuntimeError: 
undefined value torch:
  File "c:\Users\user\dev\transpose_livestream\model\TransPose\.venv\Lib\site-packages\torch\utils\_contextlib.py", line 227
        :return: Pose tensor in shape [24, 3, 3] and translation tensor in shape [3].
        """
        imu = x.repeat(self.num_total_frame, 1) if self.imu is None else torch.cat((self.imu[1:], x.view(1, -1)))
                                                                         ~~~~~ <--- HERE
        
        if self.rnn_state is not None:
'TransPoseNet.forward_online' is being compiled since it was called from 'TransPoseNet.forward'
  File "c:\Users\user\dev\transpose_livestream\model\TransPose\net.py", line 169
        data_nn = torch.cat((acc.flatten(1), ori.flatten(1)), dim=1)
    
        pose, tran = self.forward_online(data_nn)
        ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
        pose = rotation_matrix_to_axis_angle(pose.view(1, 216)).view(72)
    


In [ ]:
from torch.quantization import quantize_dynamic
quantized_model = quantize_dynamic(model, {torch.nn.LSTM, torch.nn.Linear}, dtype=torch.qint8)
torch.jit.save(torch.jit.script(quantized_model), "quantized_transpose_net.pt")